# Notes
> [Main Table of Contents](../README.md)

# In This Notebook
- Run scripts on a schedule
	- Option 1
	- Option 2
		- launchctl
			- .plist (config file for launchctl)
			- plutil (.plist file format validator)
			- Useful subcommands
			- launchctl references
			- How to use launchctl
			- launchctl troubleshooting

# Run scripts on a schedule

## Option 1

1. Wake computer from sleep
    - System Preferences -> Battery -> Schedule
        - Check Start up or wake and set frequency and time
            - Be sure to give at least 10 minutes for the computer to fully wake up before running scripts
        - Check Sleep to put the computer back to sleep

2. Use 'Automator' app to create a custom application that runs Python scripts automatically
    - Open 'Automator'
    - Select 'Application' as type of document to create
    - In the left pane, search for 'Run Shell Script' and drag it into the workflow area on the right.
    - In the 'Run Shell Script' action, change the 'Pass input' dropdown to 'as arguments.'
    - In the script area, type "python" (without quotes) followed by a space
    - Drag and drop your Python script file from Finder into the script area, after the 'python' command, this will populate the path to the python script
    - Save the Automator application by going to File > Save and giving it a name, e.g 'RunMyPythonScript'
    - Double-click the saved Automator application to run it and test that your Python script runs successfully

2. Schedule to run the custom application created by 'Automator'
    - Open 'Calendar 
    - Click '+' to create new event 
    - set start and end time 
    - Set 'Repeat' dropdown menu for frequency 
    - Set 'Alert' section to 'Custom'
        - 'Open file' from the dropdown
        - 'Choose' button to select the custom Automator application to run
        - 'OK' to save

3. Very Important
- Leave instace of VS Code & calendar app open before putting the computer to sleep
- Be sure to give at least 10 minutes for the computer to fully wake up before running scripts

4. Troubleshooting
- Automator needs access to files/folders in Desktop or some other area
    - System Preferences -> Security & Privacy -> Privacy Tab -> Unlock to make changes -> Scroll down to 'Files and Folders' in left panel -> Uncheck  Automator Desktop Folder -> Apply and close out -> Start my the beginning again except this time Check the Automator Desktop Folder and apply.  This worked today, should work again.

## Option 2 (Doesn't work well)

### launchctl
- launchctl is the interface to launchd
- launchd is a program (preferred over cron) on macOS to run scripts on a schedule
	- load, unload, and manage launch daemons and agents on macOS
- config files have extension `.plist`
- launchd further distinguishes between agents and daemons. 
	- The difference between these types are drawn from where the .plist file is saved
		- `~/Library/LaunchAgents` runs on behalf of the logged-in user
			- To restrict a script to a specific user account, use an agent.
		- `/Library/LaunchDaemons` runs on behalf of the root users
			- To run no matter who is logged in, use a daemon.

#### .plist (config file for launchctl)

In [ ]:
## plist example with useful keys

# <?xml version="1.0" encoding="UTF-8"?>
# <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
# <plist version="1.0">
# 	<dict>
# 		<key>Label</key>
# 		<string>scraperBtmBrkFinviz</string>
#     <!-- <key>RunAtLoad</key>
#     <true/> -->
#     <key>WorkingDirectory</key>
#     <string>/Users/blinkeh/Desktop</string>
# 		<key>ProgramArguments</key>
#     <array>
#     <string>/Library/Frameworks/Python.framework/Versions/3.8/bin/python3</string>
# 		<string>scraper_btmbrk/app.py</string>
#     </array>
#     <key>StartCalendarInterval</key>
#     <dict>
#     <key>Hour</key>
#     <integer>2</integer>
#     <key>Minute</key>
#     <integer>2</integer>
#     </dict>
#     <key>StandardOutPath</key>    # Log print statements
#     <string>/Users/blinkeh/Desktop/scraper_btmbrk/launchctlstdpath.out</string>
#     <key>StandardErrorPath</key>  # Log error messages
#     <string>/Users/blinkeh/Desktop/scraper_btmbrk/launchctlstdpath.err</string>
# 		<!-- <key>UserName</key>
#     <string>blinkeh</string> -->
# 	</dict>
# </plist>

: 

#### plutil
- .plist file format validator

In [ ]:
# command line usage
# plutil <pathto>.plist

: 

#### Useful subcommands
- sudo launchctl isn't a more powerful launchctl, it's just different type

Command | Description
--- | ---
launchctl dumpstate \| grep \<nameofplistfilewithoutext\>   | e.g. launchcht dumpstate | grep scraper
launchctl list \| grep scraper
launchctl load \<plistfilename\>.plist | Load script to launchd queue
launchctl unload \<plistfilename\>.plist | Remove script from launchd queue
launchctl start \<plistfilename\> | Run the script immmediately no matter what
launchctl enable system/\<plistfilename\>| Don't need this when using plist saved in `~/Library/LaunchAgents`
launchctl disable system/\<plistfilename\>|Don't need this when using plist saved in `~/Library/LaunchAgents`
id -u | Get user id

#### launchctl references
- [stackoverflow](https://stackoverflow.com/questions/15990512/launchctl-minimal-working-example-with-python)
- [brief explanation and commands](https://rakhesh.com/mac/macos-launchctl-commands/)
- [brief article](https://www.maketecheasier.com/use-launchd-run-scripts-on-schedule-macos/)


#### How to use launchctl

1. Write a xml config file with <somename>`.plist` extension
2. Save .plist file in correct directory (agent or daemon)
3. Load job into launchctl
	- `launchctl load <pathtofilename>.plist`

#### launchctl troubleshooting
- LoadFailed 5: Input/Output error
	- Usually means plist is already loaded
- If using sudo version, may need to change ownership of .plist file  

	```python
	sudo chown root /Library/LaunchDaemons/myfile.plist
	sudo chgrp wheel /Library/LaunchDaemons/myfile.plist
	and/or
	sudo chmod 600 /Library/LaunchDaemons/myfile.plist
	```